In [ ]:
from random import *

: 

In [ ]:
class Maze:
    """
    Classe Labyrinthe
    Représentation sous forme de graphe non-orienté
    dont chaque sommet est une cellule (un tuple (l,c))
    et dont la structure est représentée par un dictionnaire
      - clés : sommets
      - valeurs : ensemble des sommets voisins accessibles
    """
    def __init__(self, height : int, width : int, empty= False ):
        """
        Constructeur d'un labyrinthe de height cellules de haut
        et de width cellules de large
        Les voisinages sont initialisés à des ensembles vides quand empty vaut False
        et les voisinages comportent tout les voisins qui lui sont contigues dans la grille quand empty vaut True
        Remarque : empty vaut False par defaut et donc chaque cellule est completement emmuree par defaut.
        
        @param height : nombre de cases en hauteur
        @param width : nombre de cases en largeur
        @param empty : True si le labyrinthes est vide (pas de murs) et False si le labyrinthes est plein
        (tout les murs), empty vaut False par defaut
        """
        
        self.height    = height
        self.width     = width
        self.empty     = empty
        if empty: # creation d'un labyrinthe sans mur entre les differents sommets
            self.neighbors = {(x,y): {(x-1,y),(x,y-1),(x+1,y),(x,y+1)} for x in range(height) for y in range (width)}
            for x in range(height):
                self.neighbors[x,width-1].remove((x,width))
        else: # creation d'un labyrinthe ou les differents sommets n'ont pas de voisins (tout les murs sont presents)
            self.neighbors = {(i,j): set() for i in range(height) for j in range (width)}



    def info(self):
        """
        **NE PAS MODIFIER CETTE MÉTHODE**
        Affichage des attributs d'un objet 'Maze' (fonction utile pour deboguer)
        Retour:
            chaîne (string): description textuelle des attributs de l'objet
        """
        txt = "**Informations sur le labyrinthe**\n"
        txt += f"- Dimensions de la grille : {self.height} x {self.width}\n"
        txt += "- Voisinages :\n"
        txt += str(self.neighbors)+"\n"
        valid = True
        for c1 in {(i, j) for i in range(self.height) for j in range(self.width)}:
            for c2 in self.neighbors[c1]:
                if c1 not in self.neighbors[c2]:
                    valid = False
                    break
            else:
                continue
            break
        txt += "- Structure cohérente\n" if valid else f"- Structure incohérente : {c1} X {c2}\n"
        return txt

    def __str__(self):
        """
        **NE PAS MODIFIER CETTE MÉTHODE**
        Représentation textuelle d'un objet Maze (en utilisant des caractères ascii)
        Retour:
             chaîne (str) : chaîne de caractères représentant le labyrinthe
        """
        txt = ""
        # Première ligne
        txt += "┏"
        for j in range(self.width-1):
            txt += "━━━┳"
        txt += "━━━┓\n"
        txt += "┃"
        for j in range(self.width-1):
            txt += "   ┃" if (0,j+1) not in self.neighbors[(0,j)] else "    "
        txt += "   ┃\n"
        # Lignes normales
        for i in range(self.height-1):
            txt += "┣"
            for j in range(self.width-1):
                txt += "━━━╋" if (i+1,j) not in self.neighbors[(i,j)] else "   ╋"
            txt += "━━━┫\n" if (i+1,self.width-1) not in self.neighbors[(i,self.width-1)] else "   ┫\n"
            txt += "┃"
            for j in range(self.width):
                txt += "   ┃" if (i+1,j+1) not in self.neighbors[(i+1,j)] else "    "
            txt += "\n"
        # Bas du tableau
        txt += "┗"
        for i in range(self.width-1):
            txt += "━━━┻"
        txt += "━━━┛\n"

        return txt


    def add_wall(self, c1 : tuple, c2 : tuple) -> None:
        """
        Methode d'instance Maze permettant d'ajouter un mur entre deux cellules c1 et c2.
        
        @param c1 premiere cellule (format (x : int, y: int))
        @param c2 seconde cellule  (format (x : int, y: int))
        @return rien
        """
        
        # Facultatif : on teste si les sommets sont bien dans le labyrinthe
        assert 0 <= c1[0] < self.height and \
            0 <= c1[1] < self.width and \
            0 <= c2[0] < self.height and \
            0 <= c2[1] < self.width, \
            f"Erreur lors de l'ajout d'un mur entre {c1} et {c2} : les coordonnées de sont pas compatibles avec les dimensions du labyrinthe"
        
        # Ajout du mur
        if c2 in self.neighbors[c1]:      # Si c2 est dans les voisines de c1
            self.neighbors[c1].remove(c2) # on le retire
        if c1 in self.neighbors[c2]:      # Si c3 est dans les voisines de c2
            self.neighbors[c2].remove(c1) # on le retire
        
        return None


    def remove_wall(self,c1,c2)-> None:
        """
        Methode d'instance Maze permettant de supprimer un mur entre deux cellules c1 et c2.
        
        @param c1 premiere cellule (format (x : int, y: int))
        @param c2 seconde cellule  (format (x : int, y: int))
        @return rien
        """
        # Facultatif : on teste si les sommets sont bien dans le labyrinthe
        assert 0 <= c1[0] < self.height and \
            0 <= c1[1] < self.width and \
            0 <= c2[0] < self.height and \
            0 <= c2[1] < self.width, \
            f"Erreur lors de l'ajout d'un mur entre {c1} et {c2} : les coordonnées de sont pas compatibles avec les dimensions du labyrinthe"
        # retirer le mur
        if c2 not in self.neighbors[c1]:
            self.neighbors[c1].add(c2)
        if c1 not in self.neighbors[c2]:
            self.neighbors[c2].add(c1)
        
        return None


    def get_walls(self)-> list:
        """
        Methode d'instance Maze permettant de recuperer la liste des murs presents dans le labyrinthe.
        
        @return la liste des murs presents dans le labyrinthe (format [[(x1, y1), (x2, x2)], [(x1, y1), (x3, y3)], ...])
        """
        # initialisation de la liste des murs presents
        liste_mur=[]
        
        # parcours du labyrinthe
        for i in range(self.height):
            for j in range(self.width):
                # si il y a une cellule en dessous non voisine
                if (i+1,j) not in self.neighbors[i,j] and (i+1<self.height):
                    # alors il y a une mur
                    mur = [(i,j),(i+1,j)]
                    # si ce mur n'est pas deja recense
                    if mur not in liste_mur:
                        # on ajoute le mur a la liste
                        liste_mur.append(mur)
                # si il y a une cellule a droite non voisine
                if (i,j+1) not in self.neighbors[i,j] and (j+1<self.width):
                    # alors il y a un mur
                    mur = [(i,j),(i,j+1)]
                    # si ce mur n'est pas deja recense
                    if mur not in liste_mur:
                        # on ajoute le mur a la liste
                        liste_mur.append(mur)
                        
        # on renvoie la liste
        return liste_mur



    def fill(self)-> None:
        """
        Methode d'instance Maze permettant d'ajouter tout les murs non presents dans le labyrinthe.
        Remarque : les murs deja places sont ignores par add_wall()
        
        @return rien
        """
        
        # pour chaque emplacement de mur (parcours)
        for i in range(self.height):
            for j in range(self.width):
                
                # si on peut encore placer des murs en bas et a droite de la cellule 
                if i < (self.height-1) and j < (self.width-1):
                    # on tente de placer un mur en bas de la cellule
                    self.add_wall((i, j), (i+1, j))
                    # on tente de placer un mur a droite de la cellule
                    self.add_wall((i, j), (i, j+1))
                
                # sinon si on ne peut plus placer un mur en bas
                elif i == (self.height-1) and j < (self.width-1):
                    # on tente de placer un mur à droite de la cellule
                    self.add_wall((i, j), (i, j+1))
                
                # sinon si on ne peut plus placer un mur a droite
                elif i < (self.height-1) and j == (self.width-1):
                    # on tente de placer un mur en bas de la cellule
                    self.add_wall((i, j), (i+1, j))
                    
        return None
                    
    def to_empty(self):
        """
        Methode d'instance Maze permettant de retirer tout les murs dans le labyrinthe.
        Remarque : les murs deja absents sont ignores par remove_wall()
        Remarque 2 : l'attribut empty existe deja
        
        @return rien
        """
        # pour chaque emplacement de mur (parcours)
        for i in range(self.height):
            for j in range(self.width):
                
                # si on peut encore retirer des murs en bas et a droite de la cellule 
                if i < (self.height-1) and j < (self.width-1):
                    # on tente de retirer un mur en bas de la cellule
                    self.remove_wall((i, j), (i+1, j))
                    # on tente de retirer un mur a droite de la cellule
                    self.remove_wall((i, j), (i, j+1))
                
                # sinon si on ne peut plus retirer un mur en bas
                elif i == (self.height-1) and j < (self.width-1):
                    # on tente de retirer un mur à droite de la cellule
                    self.remove_wall((i, j), (i, j+1))
                
                # sinon si on ne peut plus retirer un mur a droite
                elif i < (self.height-1) and j == (self.width-1):
                    # on tente de retirer un mur en bas de la cellule
                    self.remove_wall((i, j), (i+1, j))
                    
        return None


    def get_contiguous_cells(self, c: tuple) -> list:
        """
        Methode d'instance Maze permettant de recuperer la liste des cellules 
        contigues a c dans la grille (sans s’occuper des eventuels murs)
        Remarque : la liste retourner ne peut pas contenir plus de quatre voisins
        
        @param c : cellule a utiliser (format (x, y))0
        @return : liste des voisins contigues de c (format [(x1, y1), (x2, y2), ...])
        """
        
        # on recupere les coordonnees de c
        i, j = c
        # on initialise la liste pour stocker les voisins
        voisins = []
        
        # si il peut y avoir un voisin en haut
        if i > 0:
            # on ajoute la cellule en haut aux voisins
            voisins.append((i-1, j))
        # si il peut y avoir un voisin en bas
        if i < self.height-2:
            # on ajoute la cellule en bas aux voisins
            voisins.append((i+1, j))
        # si il peut y avoir un voisin a gauche
        if j > 0:
            # on ajoute la cellule de gauche aux voisins
            voisins.append((i, j-1))
        # si il peut y avoir un voisin a droite
        if j < self.width-2:
            # on ajoute le voisin de droite aux voisins
            voisins.append((i, j+1))
        
        # on renvoie la liste des voisins
        return voisins


    def get_reachable_cells(self, c : tuple) -> list:
        """
        Methode d'instance Maze permettant de recuperer la liste des cellules accessibles depuis c 
        (c’est-a-dire les cellules contigues à c qui sont dans le voisinage de c)
        Remarque : la liste retournee ne peut pas contenir plus de quatre voisins
        
        @param c : cellule a utiliser (format (x, y))0
        @return : liste des cellules accessibles de c (format [(x1, y1), (x2, y2), ...])
        """
        
        # on initialise la liste des cellules accessible pour les stocker
        cell_accessible = []
        
        # on parcours les voisins de c
        for voisin in self.neighbors[c]:
            # si il n'y a pas de mur entre c et le voisin
            if voisin not in self.get_walls():
                # on ajoute le voisin a la liste
                cell_accessible.append(voisin)
        
        # on retourne la liste des cellules accessibles
        return cell_accessible

    
    def get_cells(self):
        """
        Retourne la liste de toutes les cellules de la grille du labyrinthe.
        """
        L = []
        for i in range(laby.height):
            for j in range(laby.width):
                L.append((i,j))
        return L

    
    def gen_btree(height,width):
        laby = Maze(height, width) # un labyrinthe plein (contenant tous les murs possibles)
        
        for i in range(laby.height):
            for j in range(laby.width): 
                c1 = (i,j) # parcours toute les cellules 
                voisins_EST = None
                if j<width-1:
                    voisins_EST = (i,j+1) # voisins EST
                voisins_SUD = None
                if i<height-1:
                    voisins_SUD = (i+1,j) # voisins SUD
                
                if voisins_SUD != None and voisins_EST != None: # si il y a un voisin SUD et un voisins EST
                    if not voisins_EST in laby.get_reachable_cells(c1) and not voisins_SUD in laby.get_reachable_cells(c1): # si le mur EST et SUD existe
                        aleatoire = randint(1,2)
                        if aleatoire == 1: # supprimer le mur EST
                            laby.remove_wall(c1,voisins_EST)
                        else: # supprimer le mur SUD
                            laby.remove_wall(c1,voisins_SUD)
                            
                elif voisins_EST != None and not voisins_EST in laby.get_reachable_cells(c1): # si le mur EST existe et qu'il y a un voisins EST mais pas le mur SUD, supprimer le mur EST
                    laby.remove_wall(c1,voisins_EST) # supprimer le mur EST

                elif voisins_SUD != None and not voisins_SUD in laby.get_reachable_cells(c1): # si le mur SUD existe et qu'il y a un voisins SUD mais pas le mur EST, supprimer le mur SUD
                    laby.remove_wall(c1,voisins_SUD) # supprimer le mur SUD

        return laby
    
    
    def gen_sidewinder(height,width):
        laby = Maze(height, width) # un labyrinthe plein (contenant tous les murs possibles)
        
        for i in range(height-1):
            sequence = []
            for j in range(width):
                cell = (i,j) # la cellule
                sequence.append(cell) # Ajouter la cellule à la séquence
                
                voisins_EST = None
                if j<width-1: # si le voisin EST existe
                    voisins_EST = (i,j+1) # voisins EST
                voisins_SUD = None
                if i<height-1: # si le voisin SUD existe
                    voisins_SUD = (i+1,j) # voisins SUD
                
                
                aleatoire = randint(1,2) # tirage aléatoire
                if aleatoire == 1 and voisins_EST != None:
                    laby.remove_wall(cell,voisins_EST) # Casser le mur EST de la cellule 
                    
                elif aleatoire == 2 and voisins_SUD != None:
                    cell_random = sequence[randint(0,len(sequence)-1)] # une cellule de la sequence au hasard
                    x,y = cell_random
                    laby.remove_wall(cell_random,(x+1,y)) # Casser le mur SUD de la cellule tirer au hasard
                    sequence = [] # Réinitialiser la séquence à une liste vide
                    
            sequence.append(cell) # Ajouter la dernière cellule à la séquence
            cell_random = sequence[randint(0,len(sequence)-1)] # Tirer une cellule au sort dans la séquence
            x,y = cell_random
            laby.remove_wall(cell_random,(x+1,y)) # Casser son mur SUD
        
        for i in range(width-1): # Casser tous les murs EST de la dernière ligne
            laby.remove_wall((height-1,i),(height-1,i+1))
        return laby
    
    def gen_fusion(height,width): # pas compris pour labélise les cellules de 1 à n
        laby = Maze(height, width) # un labyrinthe plein (contenant tous les murs possibles)
        label = []
        for i in range(laby.height): # on labélise les cellules de 1 à n je sais pas comment faire
            for j in range(laby.width):
                cell = (i,j)
                label.append(cell)
                
        murs = laby.get_walls() # on extrait la liste de tous les murs et on les mélange 
        random.shuffle(murs)
        #for i in murs:
            
        return laby
    
    
    def gen_exploration(height,width):
        laby = Maze(height, width) # un labyrinthe plein (contenant tous les murs possibles)
        cell = (randint(0,height-1),randint(0,width-1)) # Choisir une cellule au hasard
        cell_visite = [cell] # Marquer cette cellule comme étant visitée
        pile = [cell] # Mettre cette cellule dans sur une pile
        
        while len(pile) > 0: # Tant que la pile n’est pas vide 
            cell = pile[len(pile)-1] # Prendre la cellule en haut de la pile et l’en retirer
            del pile[len(pile)-1]
            
            voisins = laby.get_contiguous_cells(cell)
            voisin_pas_visite = []
            
            for i in voisins:
                if i not in cell_visite:
                    voisin_pas_visite.append(i) # prendre les cellules voisines qui n'ont pas été visité 
            
            if len(voisin_pas_visite) > 0: # Si cette cellule a des voisins qui n’ont pas encore été visités
                pile.append(cell) # La remettre sur la pile
                
                cell_voisine = voisin_pas_visite[randint(0,len(voisin_pas_visite)-1)] # Choisir au hasard l’une de ses cellules contigües qui n’a pas été visitée
                laby.remove_wall(cell,cell_voisine) # Casser le mur entre la cellule (celle qui a été dépilée) et celle qui vient d’être choisie
                cell_visite.append(cell_voisine) # Marquer la cellule qui vient d’être choisie comme visitée
                pile.append(cell_voisine) # Mettre la cellule voisines sur la pile
                
        return laby
    
    def gen_wilson(height,width): # boucle infini
        laby = Maze(height, width)
        cell_non_marquer = laby.get_cells()
        pos_cell = randint(0,len(cell_non_marquer)-1)
        cell_marquer = [cell_non_marquer[pos_cell]]
        del cell_non_marquer[pos_cell]
        while len(cell_non_marquer)>0:
            cell = cell_non_marquer[randint(0,len(cell_non_marquer)-1)]
            
            marche = []
            voisins = laby.get_contiguous_cells(cell)
            cell_suivante = voisins[randint(0,len(voisins)-1)]
            marche.append(cell_suivante)
            
            while cell_suivante not in cell_marquer:
                voisins = laby.get_contiguous_cells(cell_suivante)
                cell_suivante = voisins[randint(0,len(voisins)-1)]
                marche.append(cell_suivante)
                if cell_suivante in marche:
                    index = marche.index(cell_suivante)
                    while index+1<len(marche):
                        del marche[index+1]

            for i in range (len(marche)-1):
                x,y = marche[i]
                x1,y1 = marche[i+1]
                cell_marquer.append(marche[i])
                cell_non_marquer.remove(marche[i])
                laby.remove_wall(marche[i],marche[i+1])
        return laby  
    
    
    def overlay(self, content=None):
        """
        Rendu en mode texte, sur la sortie standard, \
        d'un labyrinthe avec du contenu dans les cellules
        Argument:
            content (dict) : dictionnaire tq content[cell] contient le caractère à afficher au milieu de la cellule
        Retour:
            string
        """
        if content is None:
            content = {(i,j):' ' for i in range(self.height) for j in range(self.width)}
        else:
            # Python >=3.9
            #content = content | {(i, j): ' ' for i in range(
            #    self.height) for j in range(self.width) if (i,j) not in content}
            # Python <3.9
            new_content = {(i, j): ' ' for i in range(self.height) for j in range(self.width) if (i,j) not in content}
            content = {**content, **new_content}
        txt = r""
        # Première ligne
        txt += "┏"
        for j in range(self.width-1):
            txt += "━━━┳"
        txt += "━━━┓\n"
        txt += "┃"
        for j in range(self.width-1):
            txt += " "+content[(0,j)]+" ┃" if (0,j+1) not in self.neighbors[(0,j)] else " "+content[(0,j)]+"  "
        txt += " "+content[(0,self.width-1)]+" ┃\n"
        # Lignes normales
        for i in range(self.height-1):
            txt += "┣"
            for j in range(self.width-1):
                txt += "━━━╋" if (i+1,j) not in self.neighbors[(i,j)] else "   ╋"
            txt += "━━━┫\n" if (i+1,self.width-1) not in self.neighbors[(i,self.width-1)] else "   ┫\n"
            txt += "┃"
            for j in range(self.width):
                txt += " "+content[(i+1,j)]+" ┃" if (i+1,j+1) not in self.neighbors[(i+1,j)] else " "+content[(i+1,j)]+"  "
            txt += "\n"
        # Bas du tableau
        txt += "┗"
        for i in range(self.width-1):
            txt += "━━━┻"
        txt += "━━━┛\n"
        return txt
    
    def solve_dfs(start, stop):
        
"""
Parcours du graphe jusqu’à ce qu’on trouve A

    Initialisation :
        Placer D dans la struture d’attente (file ou pile) et marquer D
        Mémoriser l’élément prédécesseur de D comme étant D
    Tant qu’il reste des cellules non-marquées :
        Prendre la « première » cellule et la retirer de la structure (appelons c, cette cellule)
        Si c correspond à A :
            C’est terminé, on a trouvé un chemin vers la cellule de destination
        Sinon :
            Pour chaque voisine de c :
                Si elle n’est pas marquée :
                    La marquer
                    La mettre dans la structure d’attente
                    Mémoriser son prédécesseur comme étant c

Reconstruction du chemin à partir des prédécesseurs

    Initialiser c à A
    Tant que c n’est pas D :
        ajouter c au chemin
        mettre le prédécesseur de c dans c
    Ajouter D au chemin

Retourner le chemin
"""
    

In [ ]:
laby = Maze(4,4, empty = True)
path = {(0, 0): '@',
        (1, 0): '*',
        (1, 1): '*',
        (2, 1): '*',
        (2, 2): '*',
        (3, 2): '*',
        (3, 3): '§'}
print(laby.overlay(path))

In [ ]:
laby = Maze.gen_btree(4, 4) #ne marche pas
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃       ┃   ┃   ┃
┣━━━╋   ╋   ╋   ┫
┃   ┃   ┃   ┃   ┃
┣   ╋   ╋   ╋   ┫
┃           ┃   ┃
┣━━━╋━━━╋   ╋   ┫
┃               ┃
┗━━━┻━━━┻━━━┻━━━┛



In [ ]:
laby = Maze(4, 4, True)
print(laby.info())

KeyError: (-1, 1)